<a href="https://colab.research.google.com/github/gjgreene/missingness-P2R/blob/main/scratchpad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%env JAX_PLATFORM_NAME=gpu
!pip install numpyro[cuda] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install pymc seaborn patsy miceforest openpyxl statsmodels NumPyro
!pip install xgboost
## exploring the imputation methods for the P2R evaluation

import pandas as pd
import numpy as np
import miceforest as mf # This import requires miceforest to be installed
from xgboost import XGBRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss
import pymc as pm
import arviz as az
import patsy


# Load dataset
df = pd.read_excel("P2R_Full_Data3.xlsx")

# Preview data
print(df.head())

df.rename(columns={
    "Type of Cancer": "Type_of_Cancer",
    "WIMD_Qunit": "WIMD_Quint"
}, inplace=True)

# Cast WIMD_Quint as ordered categorical (levels 1-5)
df["WIMD_Quint"] = pd.Categorical(
    df["WIMD_Quint"],
    categories=[1, 2, 3, 4, 5],
    ordered=True
)

# Cast Sex as nominal categorical with labels
df["Sex"] = pd.Categorical(
    df["Sex"].map({1: "Male", 2: "Female"}),
    categories=["Male", "Female"],
    ordered=False
)

# ASA Grade as ordered categorical (levels 1-4)
df["ASA_Grade"] = pd.Categorical(
    df["ASA_Grade"],
    categories=[1, 2, 3, 4],
    ordered=True
)

# Type_of_Cancer as nominal categorical
df["Type_of_Cancer"] = pd.Categorical(
    df["Type_of_Cancer"].map({1: "Colorectal", 2: "Upper_GI", 3: "HPB"}),
    categories=["Colorectal", "Upper_GI", "HPB"],
    ordered=False
)

# Cancer Stage as ordered categorical (levels 1-4)
df["Cancer_Stage"] = pd.Categorical(
    df["Cancer_Stage"],
    categories=[1, 2, 3, 4],
    ordered=True
)

# Chemotherapy as nominal categorical
df["Chemotherapy"] = pd.Categorical(
    df["Chemotherapy"].map({0: "No", 1: "Yes"}),
    categories=["No", "Yes"],
    ordered=False
)

# Radiotherapy as nominal categorical
df["Radiotherapy"] = pd.Categorical(
    df["Radiotherapy"].map({0: "No", 1: "Yes"}),
    categories=["No", "Yes"],
    ordered=False
)

# Cap unrealistic age values
df.loc[df["Age"] > 100, "Age"] = np.nan

# Create imputation flags for key variables with missingness
for col in ["Cancer_Stage", "Chemotherapy", "Radiotherapy"]:
    df[f"{col}_imputed"] = df[col].isnull().astype(int)

# Square of Age for modelling nonlinear effects
df["Age_sq"] = df["Age"] ** 2

# Define treatment and outcome clearly
treatment = "Had_P2R"
outcome = "Post_OP_LOS"

# Covariates list for future analysis
covariates = [
    "ASA_Grade", "Cancer_Stage", "Type_of_Cancer",
    "WIMD_Quint", "Time_Diagnosis_Admission", "Age",
    "Sex", "Chemotherapy", "Radiotherapy",
    "Cancer_Stage_imputed", "Chemotherapy_imputed", "Radiotherapy_imputed"
]

# Define variable types clearly
ordinal_vars = ["ASA_Grade", "Cancer_Stage"]
categorical_vars = ["Sex", "Type_of_Cancer", "WIMD_Quint", "Chemotherapy", "Radiotherapy"]
continuous_vars = ["Age", "Age_sq", "Time_Diagnosis_Admission"]

# Confirm structure and categories
print(df[covariates].dtypes)


# Check missing data summary
print(df[covariates].isnull().mean().sort_values(ascending=False))

# Quick summary statistics
print(df.describe(include='all'))

## Section 2: Imputation (MICE & GPU acceleration)

# Copy dataset for imputation to avoid modifying original
data_for_impute = df[covariates + [treatment, outcome]].copy()

# Replace infinite values with NaN
data_for_impute.replace([np.inf, -np.inf], np.nan, inplace=True)

# Store original categories to map back later
original_categories = {}
categorical_vars_all = ordinal_vars + categorical_vars

for col in categorical_vars_all:
    original_categories[col] = (df[col].cat.categories, df[col].cat.ordered)
    # Convert categorical to numeric codes for imputation
    data_for_impute[col] = df[col].cat.codes.replace(-1, np.nan).astype(float)

# Number of observed (non-missing) values for TDA
n_obs_tda = data_for_impute["Time_Diagnosis_Admission"].notna().sum()

# Drop rows with missing predictors (except TDA)
tda_predictors = data_for_impute.drop(columns=["Time_Diagnosis_Admission"]).columns
clean_data_tda = data_for_impute.dropna(subset=tda_predictors).copy()

# PMM for TDA using miceforest
tda_kernel = mf.ImputationKernel(
    data = data_for_impute.copy(),
    random_state = 42,
    num_datasets = 20,
    save_all_iterations_data = True,
    mean_match_candidates={"Time_Diagnosis_Admission": n_obs_tda}
)

# run the chained imputations
tda_kernel.mice(iterations=20)

# Extract TDA imputations
tda_imputed_values = [
    tda_kernel.complete_data(m)["Time_Diagnosis_Admission"]
    for m in range(20)
]


# Custom class to add noise for better uncertainty handling
class XGBWithNoise(XGBRegressor):
    def fit(self, X, y, **kwargs):
        super().fit(X, y, **kwargs)
        preds = super().predict(X)
        self._resid_std = np.std(y - preds, ddof=1)
        return self

    def predict(self, X, return_std=False):
        preds = super().predict(X)
        if return_std:
            return preds, np.full_like(preds, self._resid_std, dtype=float)
        return preds

# GPU-based imputer setup
gpu_xgb = XGBWithNoise(
    tree_method='hist',
    device='cuda',
    random_state=42,
    n_estimators=100,
    max_depth=6
)

imputer = IterativeImputer(
    estimator=gpu_xgb,
    sample_posterior=True,
    max_iter=20,
    random_state=42
)

# Fit imputer (includes TDA but we'll overwrite with PMM results)
imputer.fit(data_for_impute.values)

imputed_datasets = []

for m in range(20):
    imputer.random_state = 42 + m
    imputed_array = imputer.transform(data_for_impute.values)

    df_imputed = pd.DataFrame(imputed_array, columns=data_for_impute.columns)

    # Overwrite TDA with PMM-imputed values to preserve original distribution
    df_imputed["Time_Diagnosis_Admission"] = tda_imputed_values[m].values

    # Restore original categorical variables
    for col, (cats, ordered) in original_categories.items():
        codes = df_imputed[col].round().astype(int).clip(0, len(cats)-1)
        df_imputed[col] = pd.Categorical.from_codes(codes, categories=cats, ordered=ordered)

    # Reattach original treatment and outcome
    df_imputed[treatment] = df[treatment].values
    df_imputed[outcome] = df[outcome].values

    imputed_datasets.append(df_imputed)
    print(f"Imputed dataset {m+1} completed")

# Example: Check first imputed dataset
print(imputed_datasets[0].info())
print(imputed_datasets[0].describe(include='all'))

# Verify distribution of TDA against original
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.kdeplot(df["Time_Diagnosis_Admission"].dropna(), label='Original', fill=True)
sns.kdeplot(imputed_datasets[0]["Time_Diagnosis_Admission"], label='Imputed #1', fill=True)
plt.legend()
plt.title("Distribution of Time_Diagnosis_Admission: Original vs Imputed")
plt.show()

ps_covariates = [
    "Age",
    "Time_Diagnosis_Admission",
    "Sex",
    "Type_of_Cancer",
    "ASA_Grade",
    "Cancer_Stage",
    "WIMD_Quint",
    "Chemotherapy",
    "Radiotherapy",
    "Cancer_Stage_imputed",
    "Chemotherapy_imputed",
    "Radiotherapy_imputed"
]

# Helper Functions for PS and IPTW

def build_design_matrix(df, covariates):
    cat_cols = [col for col in covariates if df[col].dtype.name == "category"]
    return pd.get_dummies(df[covariates], columns=cat_cols, drop_first=True, dtype=float)

def stabilised_weights(ps, treatment):
    p_treatment = treatment.mean()
    weights = np.where(treatment == 1, p_treatment / ps, (1 - p_treatment) / (1 - ps))
    # Trim extreme weights
    lo, hi = np.percentile(weights, [1, 99])
    return np.clip(weights, lo, hi)


weighted_imputed_datasets = []

for idx, df_imp in enumerate(imputed_datasets):
    # Build design matrix
    X_ps = build_design_matrix(df_imp, ps_covariates)
    y_ps = df_imp[treatment].values

    # Standardise predictors
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_ps)

    # Logistic regression (hyperparameter tuning can be added later)
    model_ps = LogisticRegression(max_iter=5000, solver='liblinear')
    model_ps.fit(X_scaled, y_ps)

    # Predict propensity scores
    ps_scores = model_ps.predict_proba(X_scaled)[:, 1]

    # Calculate stabilised IPTW
    df_imp['ps'] = ps_scores
    df_imp['iptw'] = stabilised_weights(ps_scores, y_ps)

    weighted_imputed_datasets.append(df_imp)

    print(f"Calculated PS and IPTW for imputed dataset {idx + 1}")

def compute_smd(df, covariates, treatment, weights=None):
    smd_dict = {}
    for var in covariates:
        if df[var].dtype.name == 'category':
            levels = df[var].cat.categories
            for lvl in levels:
                mask = (df[var] == lvl).astype(int)
                mean_treated = np.average(mask[df[treatment] == 1], weights=weights[df[treatment] == 1] if weights is not None else None)
                mean_control = np.average(mask[df[treatment] == 0], weights=weights[df[treatment] == 0] if weights is not None else None)
                var_pooled = (mean_treated * (1 - mean_treated) + mean_control * (1 - mean_control)) / 2
                smd = np.abs(mean_treated - mean_control) / np.sqrt(var_pooled)
                smd_dict[f"{var}_{lvl}"] = smd
        else:
            mean_treated = np.average(df[var][df[treatment] == 1], weights=weights[df[treatment] == 1] if weights is not None else None)
            mean_control = np.average(df[var][df[treatment] == 0], weights=weights[df[treatment] == 0] if weights is not None else None)
            var_pooled = (np.var(df[var][df[treatment] == 1], ddof=1) + np.var(df[var][df[treatment] == 0], ddof=1)) / 2
            smd = np.abs(mean_treated - mean_control) / np.sqrt(var_pooled)
            smd_dict[var] = smd
    return smd_dict

# calculate weighted and unweighted SMD for each imputed dataset, then pool
smd_unweighted = []
smd_weighted = []

for df_imp in weighted_imputed_datasets:
    smd_unweighted.append(compute_smd(df_imp, covariates, treatment, weights=None))
    smd_weighted.append(compute_smd(df_imp, covariates, treatment, weights=df_imp['iptw']))

smd_unweighted = pd.DataFrame(smd_unweighted)
smd_weighted = pd.DataFrame(smd_weighted)

# Calculate the mean SMD across all imputed datasets
mean_smd_unweighted = smd_unweighted.mean()
mean_smd_weighted = smd_weighted.mean()

def plot_love_plot(smd_unweighted, smd_weighted):
    # Access index for labels (covariate names/levels)
    labels = list(smd_unweighted.index)
    # Access values attribute for the SMD values
    unweighted = list(smd_unweighted.values)
    weighted = list(smd_weighted.values)

    plt.figure(figsize=(8, len(labels) * 0.4))
    plt.hlines(labels, 0, unweighted, color='red', label='Unweighted', lw=2)
    plt.hlines(labels, 0, weighted, color='blue', label='Weighted', lw=2)
    plt.axvline(0.1, color='gray', linestyle='--')
    plt.axvline(0.2, color='black', linestyle='--')
    plt.xlabel("Standardised Mean Difference")
    plt.title("Covariate Balance (Love Plot)")
    plt.legend()
    plt.tight_layout()
    plt.show()

plot_love_plot(mean_smd_unweighted, mean_smd_weighted)


### Implementing a Negative Binomial Mixture Model
'''
# Select the first imputed dataset as an example
df_mixture = weighted_imputed_datasets[0].copy()

# Outcome variable (LOS), rounded to integers (required for NegBin)
y = df_mixture["Post_OP_LOS"].round().astype(int).values

# Define formula explicitly
formula = (
    "Had_P2R + C(ASA_Grade) + C(Cancer_Stage) + "
    "C(Type_of_Cancer) + C(WIMD_Quint) + C(Sex) + "
    "C(Chemotherapy) + C(Radiotherapy) + Age + Time_Diagnosis_Admission"
)

# Generate design matrix
X_design = patsy.dmatrix(formula, data=df_mixture, return_type='dataframe')
X_values = X_design.values

with pm.Model() as nb_mix_model:
    # Number of observations
    n = len(y)

    # Mixture component weights (two groups)
    weights = pm.Dirichlet('weights', a=np.array([1, 1]))

    # Component-specific intercepts
    intercepts = pm.Normal('intercepts', mu=0, sigma=10, shape=2)

    # Component-specific regression coefficients
    coefs = pm.Normal('coefs', mu=0, sigma=2, shape=(2, X_values.shape[1]))

    # Component-specific dispersion parameters
    alphas = pm.HalfCauchy('alphas', beta=2, shape=2)

    # Linear combination and expected LOS (mu) for each mixture component
    mu_components = pm.math.exp(intercepts[:, None] + pm.math.dot(coefs, X_values.T))

    # Mixture Negative Binomial Likelihood
    y_obs = pm.Mixture(
        'y_obs',
        w=weights,
        comp_dists=[
            pm.NegativeBinomial.dist(mu=mu_components[0], alpha=alphas[0]),
            pm.NegativeBinomial.dist(mu=mu_components[1], alpha=alphas[1])
        ],
        observed=y
    )

    # Sample from the posterior
    trace_mix = pm.sample(draws=2000, tune=2000, target_accept=0.95, random_seed=42)

az.plot_trace(trace_mix, var_names=['weights', 'intercepts', 'alphas'])
plt.tight_layout()
plt.show()

az.summary(trace_mix, var_names=['weights', 'intercepts', 'alphas'])

with nb_mix_model:
    ppc = pm.sample_posterior_predictive(trace_mix, random_seed=42)

y_pred = ppc.posterior_predictive['y_obs'].stack(draws=("chain", "draw")).values

plt.figure(figsize=(8, 5))
sns.kdeplot(y, label="Observed LOS", fill=True, color="gray")
sns.kdeplot(y_pred.mean(axis=0), label="Predicted LOS", fill=True, color="blue", alpha=0.6)
plt.legend()
plt.title("Posterior Predictive Check for Mixture Model")
plt.xlabel("Length of Stay")
plt.show()

weights_summary = az.summary(trace_mix, var_names=['weights'])
print(weights_summary)
'''

import pymc as pm
import arviz as az
import pytensor.tensor as pt               # PyMC ≥5 uses pytensor
import numpy as np
import pandas as pd

def build_X(df, formula):
    import patsy
    X_df = patsy.dmatrix(formula, df, return_type="dataframe")
    return X_df, list(X_df.columns)

def fit_mix_nb_pymc(df_imp, formula, rng_seed=42):
    y = df_imp["Post_OP_LOS"].round().astype(int).values
    X_df, colnames = build_X(df_imp, formula)
    X = X_df.values
    n_obs = y.size

    with pm.Model() as model:
        w     = pm.Dirichlet("w", a=np.ones(2))
        beta0 = pm.Normal("beta0",  0, 10, shape=2)
        beta  = pm.Normal("beta",   0,  2, shape=(2, X.shape[1]))
        alpha = pm.HalfCauchy("α",  2,       shape=2)

        mu = pt.exp(beta0[:, None] + beta.dot(X.T))     # (2, n_obs)

        comp = [
            pm.NegativeBinomial.dist(mu=mu[0], alpha=alpha[0]),
            pm.NegativeBinomial.dist(mu=mu[1], alpha=alpha[1])
        ]

        pm.Mixture("y_like", w=w, comp_dists=comp, observed=y)

        idata = pm.sample(draws=2000, tune=1000,
                          chains=2, target_accept=0.9,
                          random_seed=rng_seed,
                          init="jitter+adapt_diag")
    return idata, colnames

import os

# Define formula explicitly
formula = (
    "Had_P2R + C(ASA_Grade) + C(Cancer_Stage) + "
    "C(Type_of_Cancer) + C(WIMD_Quint) + C(Sex) + "
    "C(Chemotherapy) + C(Radiotherapy) + Age + Time_Diagnosis_Admission"
)

posterior_list = []
for m, df_imp in enumerate(weighted_imputed_datasets):
    idata, colnames = fit_mix_nb_pymc(df_imp, formula, rng_seed=2025 + m)
    posterior_list.append(idata.posterior)
    print(f"✓ finished imputation {m+1}/20")

combined_posterior = az.concat(posterior_list, dim="draw")
print(az.summary(combined_posterior, var_names=["beta"], filter_vars="like")
        .loc[:, ["mean", "hdi_3%", "hdi_97%"]])


# quick summary of the treatment effect coefficients
az.summary(combined_posterior,
           var_names=["beta"], filter_vars="like").loc[:, ["mean", "hdi_3%", "hdi_97%"]]


import arviz as az
import pandas as pd
from google.colab import files      # ⇠ provides files.download()

# ⬛ 1. create the coefficient table ---------------------------------------
summary = (
    az.summary(
        combined_posterior,
        var_names=["beta"],          # only regression coefficients
        filter_vars="like",
        round_to=4
    )[["mean", "hdi_3%", "hdi_97%"]]
    .reset_index()
    .rename(columns={
        "index":    "parameter",
        "mean":     "posterior_mean",
        "hdi_3%":   "lower_95",
        "hdi_97%":  "upper_95"
    })
)

# ⬛ 2. save to Excel inside Colab’s scratch disk ---------------------------
out_path = "/mnt/data/mixture_nb_coefficients.xlsx"
summary.to_excel(out_path, index=False)
print(f"Excel file saved to {out_path}")

# ⬛ 3. download the file to your computer ---------------------------------
files.download(out_path)             # a browser download dialogue pops up

# ⬛ 4. (optional) preview in-notebook -------------------------------------
import ace_tools as tools
tools.display_dataframe_to_user(name="Mixture NB coefficients", dataframe=summary)
